In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

%matplotlib inline

In [2]:
EPOCHS = 1
BATCH_SIZE = 5
VAL_STEPS = 5000

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
device = torch.device('cpu')

In [4]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [6]:
# def plot_img (img):
#     plt.imshow(img.numpy()[0], cmap='gray')
    
# data_iter = iter(trainloader)
# images, labels = data_iter.next()

In [7]:
class Net (nn.Module):
    
    def __init__ (self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
        
    def forward (self, x):
        
        # Conv 1
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        
        # Conv2
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Flatten
        x = x.view(-1, 64*7*7)
        
        # Fc1
        x = F.relu(self.fc1(x))
        
        # Fc2
        x = self.fc2(x)
        
        return x
        

In [8]:
net = Net()

In [9]:
net.fc1.in_features

3136

In [10]:
net = Net()

net.to(device)

net = nn.DataParallel(net)

In [11]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)

In [12]:
step = 0
val_iterator = iter(cycle(testloader))

for epoch in range(EPOCHS):
    
    for data in trainloader:
        
        net.train()
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/BATCH_SIZE
        
        if step % 50 == 0:
            print('Epoch: %d, Step: %5d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
        
        if step % 100 == 0:
            
            net.eval()
            
            val_acc = 0
            
            with torch.no_grad():
                for s in range(VAL_STEPS):
                    val_x, val_y = next(val_iterator)
                    
                    val_x, val_y = val_x.to(device), val_y.to(device)
                    
                    val_logits = net(val_x)
                    _, val_preds = torch.max(val_logits, 1)

                    val_correct = (val_preds == val_y).sum().item()
                    
                    val_acc += val_correct / BATCH_SIZE
            
            print('Validation loss: %.3f' % (val_acc / VAL_STEPS))

Epoch: 0, Step:    50, Minibatch loss: 1.734, Minibatch accuracy: 0.400
Epoch: 0, Step:   100, Minibatch loss: 0.500, Minibatch accuracy: 0.800
Validation loss: 0.723
Epoch: 0, Step:   150, Minibatch loss: 1.030, Minibatch accuracy: 0.600
Epoch: 0, Step:   200, Minibatch loss: 0.110, Minibatch accuracy: 1.000
Validation loss: 0.898
Epoch: 0, Step:   250, Minibatch loss: 0.236, Minibatch accuracy: 1.000
Epoch: 0, Step:   300, Minibatch loss: 0.326, Minibatch accuracy: 0.800


Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zhmiao/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zhmiao/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zhmiao/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhmiao/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhmiao/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/zhmiao/miniconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/zhmiao/miniconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = s

KeyboardInterrupt: 

In [12]:
# Class testing

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

In [13]:
data = next(iter(trainloader))

In [14]:
images, labels = data
images, labels = images.to(device), labels.to(device)
outputs = net(images)
_, predicted = torch.max(outputs, 1)

In [15]:
predicted

tensor([ 8,  8,  8,  5,  5], device='cuda:0')

In [16]:
labels

tensor([ 2,  9,  7,  7,  1], device='cuda:0')

In [17]:
predicted == labels

tensor([ 0,  0,  0,  0,  0], dtype=torch.uint8, device='cuda:0')

In [20]:
c = (predicted == labels).squeeze()

In [23]:
torch.tensor([0, 1, 1, 0, 0]).squeeze()

tensor([ 0,  1,  1,  0,  0])

In [21]:
label = labels[0]
class_correct[label] += 

tensor([ 0,  0,  0,  0,  0], dtype=torch.uint8, device='cuda:0')

In [14]:
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

RuntimeError: Expected object of type torch.cuda.LongTensor but found type torch.LongTensor for argument #2 'other'

In [ ]:
classes[i], 100 * class_correct[i] / class_total[i]))

In [17]:
# Testing

correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        logits = net(images)
        _, predictions = torch.max(logits, 1)
        total += labels.shape[0]
        correct += (predictions == labels).sum().item()
        
print(correct/total)

0.9852


In [16]:
labels.shape[0]

128

$(A \times 1 \times B \times C \times 1 \times D)$
$(A \times B \times C \times D)$